In [93]:
using Flux, Images, MLDatasets, Plots, RDatasets

using Flux: crossentropy, onecold, onehotbatch, params

using Random, Statistics, Distributions

using Turing, FillArrays, ReverseDiff, LinearAlgebra

using Gen

using DataFrames

using MLLabelUtils

In [94]:
Random.seed!(5)

TaskLocalRNG()

In [95]:
function split(X, y::AbstractVector; dims=1, ratio_train=0.8, kwargs...)
    n = length(y)
    size(X, dims) == n || throw(DimensionMismatch("..."))

    n_train = round(Int, ratio_train*n)
    i_rand = randperm(n)
    i_train = i_rand[1:n_train]
    i_test = i_rand[n_train+1:end]

    return selectdim(X, dims, i_train), y[i_train], selectdim(X, dims, i_test), y[i_test]
end

function onehot(y, classes)
    y_onehot = falses(length(classes), length(y))
    for (i, class) in enumerate(classes)
        y_onehot[i, y .== class] .= 1
    end
    return y_onehot
end

function prepare_data(X, y; do_normal=true, do_onehot=true, kwargs...)
    X_train, y_train, X_test, y_test = split(X, y; kwargs...)

    if do_normal
        X_train, X_test = normalize(X_train, X_test; kwargs...)
    end

    classes = unique(y)

    if do_onehot
        y_train = onehot(y_train, classes)
        y_test = onehot(y_test, classes)
    end

    return X_train, y_train, X_test, y_test, classes
end

function onehot_number(number, classes)
    return classes .== number
end

onehot_number (generic function with 1 method)

In [217]:
iris = dataset("datasets", "iris")

X = Matrix(iris[:, 1:4])
y = iris.Species

X_train, y_train, X_test, y_test, classes = prepare_data(X', y; do_onehot=false, do_normal=false, dims=2)

([5.9 6.5 … 5.8 5.2; 3.0 2.8 … 2.7 3.4; 4.2 4.6 … 5.1 1.4; 1.5 1.5 … 1.9 0.2], CategoricalArrays.CategoricalValue{String, UInt8}["versicolor", "versicolor", "virginica", "setosa", "setosa", "versicolor", "versicolor", "setosa", "versicolor", "versicolor"  …  "versicolor", "setosa", "versicolor", "versicolor", "versicolor", "setosa", "setosa", "setosa", "virginica", "setosa"], [5.7 6.5 … 6.7 4.9; 2.6 3.0 … 3.1 3.1; 3.5 5.8 … 5.6 1.5; 1.0 2.2 … 2.4 0.2], CategoricalArrays.CategoricalValue{String, UInt8}["versicolor", "virginica", "setosa", "virginica", "setosa", "setosa", "virginica", "virginica", "virginica", "setosa"  …  "virginica", "virginica", "versicolor", "versicolor", "versicolor", "virginica", "versicolor", "setosa", "virginica", "setosa"], ["setosa", "versicolor", "virginica"])

In [221]:
y_train

120-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "versicolor"
 "versicolor"
 "virginica"
 "setosa"
 "setosa"
 "versicolor"
 "versicolor"
 "setosa"
 "versicolor"
 "versicolor"
 "virginica"
 "versicolor"
 "versicolor"
 ⋮
 "versicolor"
 "setosa"
 "versicolor"
 "setosa"
 "versicolor"
 "versicolor"
 "versicolor"
 "setosa"
 "setosa"
 "setosa"
 "virginica"
 "setosa"

In [131]:
classes = unique(y)
classes_numbers = [1,2,3]
isequal(onecold(onehot(y, classes), classes), y)

encoding = Dict()
for (i, class) in enumerate(classes)
    encoding[class] = i
end

y_encoded = Int8[]
for class in y
    push!(y_encoded,encoding[class])
end

y_train_encoded = Int8[]
for class in y_train
    push!(y_train_encoded,encoding[class])
end

y_test_encoded = Int8[]
for class in y_test
    push!(y_test_encoded,encoding[class])
end

y_train=y_train_encoded
y_test=y_test_encoded;

LoadError: KeyError: key 1 not found

In [82]:
y_encoded = Int8[]
for class in y
    push!(y_encoded,encoding[class])
end

In [99]:
# flatten input data

X_train = Flux.flatten(X_train)

X_test = Flux.flatten(X_test)

4×30 reshape(view(adjoint(::Matrix{Float64}), :, [85, 65, 50, 72, 22, 78, 110, 46, 48, 36  …  133, 145, 26, 126, 1, 54, 88, 68, 39, 77]), 4, 30) with eltype Float64:
 5.4  5.6  5.0  6.1  5.1  6.7  7.2  4.8  …  7.2  5.1  5.5  6.3  5.8  4.4  6.8
 3.0  2.9  3.3  2.8  3.7  3.0  3.6  3.0     3.2  3.5  2.3  2.3  2.7  3.0  2.8
 4.5  3.6  1.4  4.0  1.5  5.0  6.1  1.4     6.0  1.4  4.0  4.4  4.1  1.3  4.8
 1.5  1.3  0.2  1.3  0.4  1.7  2.5  0.3     1.8  0.2  1.3  1.3  1.0  0.2  1.4

In [100]:
#Network for multiclass classification
nn_initial = Chain(
    Dense(4, 16, relu),
    Dense(16, 3),
    softmax
)

Chain(
  Dense(4 => 16, relu),                 # 80 parameters
  Dense(16 => 3),                       # 51 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 131 parameters, 780 bytes.

In [101]:
###########################TESTING###################################

# Extract weights and a helper function to reconstruct NN from weights
parameters_initial, reconstruct = Flux.destructure(nn_initial)

nb_param = length(parameters_initial) # number of paraemters in NN

131

In [102]:
#nn_initial(X_train[:,1])
#y_onehot = falses(length(classes))
#y_onehot[y[1]] = y[1]

In [103]:
@gen function test(nn, input)
    prob = nn(input)
    obs_test = {:obs_test} ~ categorical(prob)
    print(obs_test)
    return obs_test
end
test(nn_initial,X_train[:,1])

2

2

In [71]:
#multiclass classification:
@gen function bayes_nn(xs, nparameters, reconstruct)
    # Create the weight and bias vector.
    parameters = Float32[]
    for i=1:nparameters
        push!(parameters, {(:parameters, i)} ~ normal(0,1))
    end

    # Construct NN from parameters
    nn = reconstruct(parameters)
    # Forward NN to make predictions
    # Observe each prediction.
    obs = Int64[]
    for i=1:size(xs)[2]
        #probs = convert(Vector{Float64}, nn(xs[:,i]))
        prob = nn(xs[:,i])
        observation = {(:obs, i)} ~ categorical(prob)
        push!(obs, observation)
    end
    obs
end;

In [224]:
function make_constraints()
    constraints = Gen.choicemap()
    for i=1:size(X_train)[2]
        constraints[(:obs, i)] = y_train[i]
    end
    constraints
end

function get_params_from_trace(trace, nb_params)
    params = Float32[]
    for i=1:nb_params
        push!(params, trace[(:parameters,i)])
    end
    return params
end

observations = make_constraints();

In [107]:
# function do_inference(amount_of_computation, iter)
#     (trace, _) = Gen.importance_resampling( bayes_nn,
#                                             (X_train, nb_param, reconstruct),
#                                             observations,
#                                             amount_of_computation
#                                             );
#     println(iter)
#     return trace
# end

In [128]:
function block_resimulation_update(tr)
    for i=1:tr.args[2]
        latent_variable = Gen.select((:parameters,i))
        (tr, _) = mh(tr, latent_variable)
    end
    tr
end

function block_resimulation_inference(tr, n_samples)
    trs = []
    for iter=1:n_samples
        tr = block_resimulation_update(tr)
        trace_params = get_params_from_trace(tr, nb_param)
        current_nn = tr.args[3](trace_params)
        y_hat_pred = Int32[]
        for i = 1:size(X_test)[2]
            push!(y_hat_pred, argmax(current_nn(X_test[:,i])))
        end

        
        println("Accuracy for current trace $(iter): $(mean(y_hat_pred .== y_test))")
        push!(trs, tr)
    end
    
    trs
end;

In [129]:
(trace, _) = Gen.generate(bayes_nn, (X_train, nb_param, reconstruct), observations);
trs = block_resimulation_inference(trace, 10)

Accuracy for current trace 1: 0.5666666666666667
Accuracy for current trace 2: 0.9
Accuracy for current trace 3: 0.9333333333333333
Accuracy for current trace 4: 0.9333333333333333
Accuracy for current trace 5: 0.9
Accuracy for current trace 6: 0.9666666666666667
Accuracy for current trace 7: 0.9666666666666667
Accuracy for current trace 8: 1.0
Accuracy for current trace 9: 1.0
Accuracy for current trace 10: 0.9666666666666667


10-element Vector{Any}:
 Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any, Any, Any], false, Union{Nothing, Some{Any}}[nothing, nothing, nothing], var"##bayes_nn#534", Bool[0, 0, 0], false), Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}((:parameters, 114) => Gen.ChoiceOrCallRecord{Float64}(-1.779214352758611, -2.5017403897357946, NaN, true), (:parameters, 37) => Gen.ChoiceOrCallRecord{Float64}(-0.9012392855209004, -1.3250546580877842, NaN, true), (:parameters, 107) => Gen.ChoiceOrCallRecord{Float64}(-1.0506894552540147, -1.4709126988956618, NaN, true), (:obs, 51) => Gen.ChoiceOrCallRecord{Int64}(3, -0.0396010387122467, NaN, true), (:obs, 34) => Gen.ChoiceOrCallRecord{Int64}(1, -3.1406317522393983, NaN, true), (:obs, 53) => Gen.ChoiceOrCallRecord{Int64}(2, -0.3812016277210727, NaN, true), (:obs, 73) => Gen.ChoiceOrCallRecord{Int64}(1, -2.216166782509147, NaN, true), (:parameters, 30) => Gen.Choi

In [ ]:
X=X'

In [213]:
#argmax(nn_initial(X),dims=1)
model = Chain(
    Dense(4, 16, relu),
    Dense(16, 3),
    softmax
)

Chain(
  Dense(4 => 16, relu),                 # 80 parameters
  Dense(16 => 3),                       # 51 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 131 parameters, 780 bytes.

In [214]:
y_train_encoded = onehotbatch(y_train, 1:3)

y_test_encoded = onehotbatch(y_test, 1:3)

3×30 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  1  ⋅  1  ⋅  ⋅  1  1  1  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  1  ⋅
 1  1  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  1  1     1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  1  1  ⋅  1
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅     ⋅  1  1  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

In [215]:
# define loss function
loss(x, y) = crossentropy(model(x), y)

# track parameters
ps = params(model)

# select optimizer
learning_rate = Float32(0.01)

opt = Flux.Adam(learning_rate)

Adam(0.009999999776482582, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [216]:
# train model

loss_history = []

epochs = 100

for epoch in 1:epochs
    # train model
    Flux.train!(loss, ps, [(X_train, y_train_encoded)], opt)
    # print report
    train_loss = loss(X_train, y_train_encoded)
    push!(loss_history, train_loss)
    
    
    y_hat_pred = Int32[]
    for i = 1:size(X_test)[2]
        push!(y_hat_pred, argmax(model(X_test[:,i])))
    end
    println("Accuracy epoch $(epoch): $(mean(y_hat_pred .== y_test))")
    #println("Epoch = $epoch : Training Loss = $train_loss")
end

Accuracy epoch 1: 0.1
Accuracy epoch 2: 0.3333333333333333
Accuracy epoch 3: 0.36666666666666664
Accuracy epoch 4: 0.3333333333333333
Accuracy epoch 5: 0.3
Accuracy epoch 6: 0.3
Accuracy epoch 7: 0.3
Accuracy epoch 8: 0.3
Accuracy epoch 9: 0.3333333333333333
Accuracy epoch 10: 0.6
Accuracy epoch 11: 0.6333333333333333
Accuracy epoch 12: 0.6333333333333333
Accuracy epoch 13: 0.6333333333333333
Accuracy epoch 14: 0.6
Accuracy epoch 15: 0.6333333333333333
Accuracy epoch 16: 0.7
Accuracy epoch 17: 0.7
Accuracy epoch 18: 0.7
Accuracy epoch 19: 0.7
Accuracy epoch 20: 0.7
Accuracy epoch 21: 0.7
Accuracy epoch 22: 0.6666666666666666
Accuracy epoch 23: 0.6
Accuracy epoch 24: 0.6333333333333333
Accuracy epoch 25: 0.6333333333333333
Accuracy epoch 26: 0.6333333333333333
Accuracy epoch 27: 0.6333333333333333
Accuracy epoch 28: 0.6333333333333333
Accuracy epoch 29: 0.6333333333333333
Accuracy epoch 30: 0.6333333333333333
Accuracy epoch 31: 0.6333333333333333
Accuracy epoch 32: 0.6333333333333333
Ac

In [205]:
# make predictions
y_hat_pred = Int32[]
for i = 1:size(X_test)[2]
    push!(y_hat_pred, argmax(model(X_test[:,i])))
end
println("Accuracy: $(mean(y_hat_pred .== y_test))")

Accuracy: 0.9666666666666667


In [14]:
# make predictions

y_hat_raw = model(X_test)

y_hat = onecold(y_hat_raw) .- 1

y = y_test_raw

mean(y_hat .== y)


0.9597

In [206]:
# initialize plot

gr(size = (600, 600))

# plot learning curve

p_l_curve = plot(1:epochs, loss_history,
    xlabel = "Epochs",
    ylabel = "Loss",
    title = "Learning Curve",
    legend = false,
    color = :blue,
    linewidth = 2
)

# save plot

savefig(p_l_curve, "ann_learning_curve.svg")

"C:\\Facultate\\Master\\An 1\\Sem1\\Probabilistic Programming\\Project 2\\ann_learning_curve.svg"